### Excercise 1.1.1

In [1]:
import nltk
import random
import numpy as np
import scipy as sp
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [2]:
# I just called it more (MOvie REviews) to write less
from nltk.corpus import movie_reviews as more

In [3]:
raw_movie_docs = [
    (more.raw(fileid), category) for category in more.categories()
    for fileid in more.fileids(category)]

In [4]:
random.seed(15)
random.shuffle(raw_movie_docs)
# The first 200 random documents will be for final testing
movie_test = raw_movie_docs[:200]
# The last 1800 documents (we know there are 2000 documents) will be for training
movie_dev = raw_movie_docs[200:]

In [5]:
# 1600 docs for development test data
train_data = movie_dev[200:]
# 200 docs for train-data
dev_test_data = movie_dev[:200]

In [6]:
# lists of the texts and targets (positive og negative) from train_data
train_texts = []
train_target = []
for review in train_data:
    text, sentiment = review
    train_texts.append(text)
    train_target.append(sentiment)

In [7]:
# lists of the texts and targets (positive og negative) from dev_test_data
dev_test_texts = []
dev_test_target = []
for review in dev_test_data:
    text, sentiment = review
    dev_test_texts.append(text)
    dev_test_target.append(sentiment)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
# Considers the whole set of training data, to determine which features to extract
v = CountVectorizer()
v.fit(train_texts)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [10]:
# Extract features from the training data and the test data
train_vectors = v.transform(train_texts)
dev_test_vectors = v.transform(dev_test_texts)

In [11]:
# Train classifier
clf = MultinomialNB()
clf.fit(train_vectors, train_target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [12]:
# predict one 
#print(dev_test_texts[14])
#clf.predict(dev_test_vectors[14])
# Predict all
#clf.predict(dev_test_vectors)

In [13]:
# Get the accuracy of the multinomial naive Bayes text classifier
clf.score(dev_test_vectors, dev_test_target)

0.81

### Exercise 1.1.2

In [14]:
def multi_nb_exp(train_data, test_data):
    def separate(data):
        texts = []
        targets = []
        for review in data:
            text, sentiment = review
            texts.append(text)
            targets.append(sentiment)
        return (texts, targets)
    train_texts, train_target = separate(train_data)
    test_texts, test_target = separate(test_data)
    
    v = CountVectorizer()
    v.fit(train_texts)
    
    train_vectors = v.transform(train_texts)
    test_vectors = v.transform(test_texts)
    
    
    clf = MultinomialNB()
    clf.fit(train_vectors, train_target)
    
    prediction = clf.predict(test_vectors)
    accuracy = clf.score(test_vectors, test_target)
    
    return (accuracy, prediction)

In [15]:
(acc, pred) = multi_nb_exp(train_data, dev_test_data)
print("acc:", acc)

acc: 0.81


### Exercise 1.1.3

In [16]:
import math
def n_fold_kwargs(experiment, dev_data, folds=10, **kwargs):
    # Assume that the data can be devided by folds into equal folds.
    size = len(dev_data)/folds
    acc_sum = 0
    sum_squared_diff = 0
    accuracies = []
    data_folds = []
    for x in range(folds):
        data_folds.append(dev_data[(x*int(size)):((x+1)*int(size))])
    
    for run in range(folds):
        train_tmp = [k for k in data_folds if k is not data_folds[run]]
        test = data_folds[run]
        train = []
        for x in train_tmp:
            train = train + x
        (acc, pred) = experiment(train, test)
        accuracies.append(acc)
        acc_sum = acc_sum + acc
    mean = acc_sum/folds
    
    for x in accuracies:
        sum_squared_diff = sum_squared_diff + ((x-mean)*(x-mean))
    standard_deviation = math.sqrt(sum_squared_diff/(folds-1))
    
    return (accuracies, mean, standard_deviation)

In [17]:
(acc, mean, standard) = n_fold_kwargs(multi_nb_exp, movie_dev, folds=9)
print("acc:", acc)
print("mean:", mean)
print("Standard:", standard)

acc: [0.81, 0.815, 0.81, 0.75, 0.815, 0.815, 0.81, 0.82, 0.82]
mean: 0.8072222222222223
Standard: 0.021810420547170657


### Exercise 1.2

_Note_:
If I understand this exercise correctly, I'm supposed to use the \*\*kwargs to pass in ngram_range= and binary=.
I was not able to do this. Could not figure out how. That is why I made a new function that takes them in directly.

In [18]:
def multi_nb_experiment(train_data, test_data, ngram_range, binary):
    def separate(data):
        texts = []
        targets = []
        for review in data:
            text, sentiment = review
            texts.append(text)
            targets.append(sentiment)
        return (texts, targets)
    train_texts, train_target = separate(train_data)
    test_texts, test_target = separate(test_data)
    
    v = CountVectorizer(ngram_range=ngram_range, binary=binary)
    v.fit(train_texts)
    
    train_vectors = v.transform(train_texts)
    test_vectors = v.transform(test_texts)
    
    
    clf = MultinomialNB()
    clf.fit(train_vectors, train_target)
    
    prediction = clf.predict(test_vectors)
    accuracy = clf.score(test_vectors, test_target)
    
    return (accuracy, prediction)

def n_folds(experiment, dev_data, ngram_range, binary, folds=10):
    # Assume that the data can be devided by folds into equal folds.
    size = len(dev_data)/folds
    acc_sum = 0
    sum_squared_diff = 0
    accuracies = []
    data_folds = []
    for x in range(folds):
        data_folds.append(dev_data[(x*int(size)):((x+1)*int(size))])
    
    for run in range(folds):
        train_tmp = [k for k in data_folds if k is not data_folds[run]]
        test = data_folds[run]
        train = []
        for x in train_tmp:
            train = train + x
        (acc, pred) = experiment(train, test, ngram_range, binary)
        accuracies.append(acc)
        acc_sum = acc_sum + acc
    mean = acc_sum/folds
    
    for x in accuracies:
        sum_squared_diff = sum_squared_diff + ((x-mean)*(x-mean))
    standard_deviation = math.sqrt(sum_squared_diff/(folds-1))
    
    return (accuracies, mean, standard_deviation)

In [19]:
highest = 0

(acc1, mean1, standard1) = n_folds(multi_nb_experiment, movie_dev, folds=9, ngram_range=[1,1], binary=True)
if max(acc1) > highest:
    highest = max(acc1)

(acc2, mean2, standard2) = n_folds(multi_nb_experiment, movie_dev, folds=9, ngram_range=[1,2], binary=True)
if max(acc2) > highest:
    highest = max(acc2)

(acc3, mean3, standard3) = n_folds(multi_nb_experiment, movie_dev, folds=9, ngram_range=[1,3], binary=True)
if max(acc3) > highest:
    highest = max(acc3)

(acc4, mean4, standard4) = n_folds(multi_nb_experiment, movie_dev, folds=9, ngram_range=[1,1], binary=False)
if max(acc4) > highest:
    highest = max(acc4)

(acc5, mean5, standard5) = n_folds(multi_nb_experiment, movie_dev, folds=9, ngram_range=[1,2], binary=False)
if max(acc5) > highest:
    highest = max(acc5)

(acc6, mean6, standard6) = n_folds(multi_nb_experiment, movie_dev, folds=9, ngram_range=[1,3], binary=False)
if max(acc6) > highest:
    highest = max(acc6)

print("highest:", highest)
# It seems that boolean=True and ngram_range=[1,3] yields the best accuracy.

highest: 0.88


In [20]:
# help(CountVectorizer)

### Exercise 1.3

In [21]:
def multi_LR_experiment(train_data, test_data, ngram_range, binary):
    def separate(data):
        texts = []
        targets = []
        for review in data:
            text, sentiment = review
            texts.append(text)
            targets.append(sentiment)
        return (texts, targets)
    train_texts, train_target = separate(train_data)
    test_texts, test_target = separate(test_data)
    
    v = CountVectorizer(ngram_range=ngram_range, binary=binary)
    v.fit(train_texts)
    
    train_vectors = v.transform(train_texts)
    test_vectors = v.transform(test_texts)
    
    
    clf = LogisticRegression(random_state=0, solver='lbfgs')
    clf.fit(train_vectors, train_target)
    
    prediction = clf.predict(test_vectors)
    accuracy = clf.score(test_vectors, test_target)
    
    return (accuracy, prediction)




(acc, mean, standard) = n_folds(multi_LR_experiment, movie_dev, folds=9, ngram_range=[1,3], binary=True)
print("Acc:", acc)
print("mean:", mean)
print("Standard:", standard)

Acc: [0.855, 0.83, 0.87, 0.85, 0.885, 0.86, 0.865, 0.89, 0.845]
mean: 0.8611111111111112
Standard: 0.019002923751652315


### Exercise 1.4.1

In [22]:
documents = [(more.words(fileid), category)
             for category in more.categories()
             for fileid in more.fileids(category)]

random.seed(15)
random.shuffle(documents)

# Last verification
doc_test = documents[:200]
# Testing in dev
doc_dev_test = documents[200:400]
# Trainingset
doc_train = documents[400:]

In [23]:
def bernoulli_exp_p(train_data, test_data, feature_numbs=2000):
    all_words = nltk.FreqDist(w.lower() for w in more.words())
    word_features = list(all_words)[:feature_numbs]
                                
    def document_features(document): 
        document_words = set(document) 
        features = {}
        for word in word_features:
            features['contains({})'.format(word)] = (word in document_words)
        return features
    
    train_data = [(document_features(document), label) for (document, label) in train_data]
    test_data  = [(document_features(document), label) for (document, label) in test_data]
    
    def seperate(data):
        texts = []
        targets = []
        for t, f in data:
            texts.append(t)
            targets.append(f)
        return texts, targets

    vec = DictVectorizer()
    
    train, train_target = seperate(train_data)
    test, test_target   = seperate(test_data)

    vec.fit(train)
    
    train_vec = vec.transform(train)
    test_vec  = vec.transform(test)

    clf = BernoulliNB()
    clf.fit(train_vec, train_target)
    
    pred = clf.predict(test_vec)
    score = clf.score(test_vec, test_target)

    return pred, score

In [24]:
def n_fold_n(experiment, dev_data, folds=9, feature_numbs=2000):
    
    size  = len(dev_data)/folds
    acc_sum = 0
    sum_squared_diff = 0
    accuracies = []
    data_folds = []
    
    for x in range(folds):
        data_folds.append(dev_data[(x*int(size)):((x+1)*int(size))])
    
    for run in range(folds):
        train_tmp = [data_folds[x] for x in range(len(data_folds)) if x != run]
        test = data_folds[run]
        train = []
        for x in train_tmp:
            train = train + x
        (pred, acc) = experiment(train, test, feature_numbs)
        accuracies.append(acc)
        acc_sum = acc_sum + acc
        
    mean = acc_sum/folds
    
    for x in accuracies:
        sum_squared_diff = sum_squared_diff + ((x-mean)*(x-mean))
    standard_deviation = math.sqrt(sum_squared_diff/(folds-1))
    
    return (accuracies, mean, standard_deviation)

### Exercise 1.4.2

In [25]:
data = doc_train + doc_dev_test 

accuracies, mean, std = n_fold_n(bernoulli_exp_p, data)

print(mean, std)

0.8061111111111111 0.03533569174519031


### Exercise 1.5.1

In [26]:
def logisticReg_exp(train_data, test_data, feature_numbs=2000):
    all_words = nltk.FreqDist(w.lower() for w in more.words())
    word_features = list(all_words)[:feature_numbs]
                                
    def document_features(document): 
        document_words = set(document) 
        features = {}
        for word in word_features:
            features['contains({})'.format(word)] = (word in document_words)
        return features
    
    train_data = [(document_features(document), label) for (document, label) in train_data]
    test_data  = [(document_features(document), label) for (document, label) in test_data]
    
    def seperate(data):
        texts = []
        targets = []
        for t, f in data:
            texts.append(t)
            targets.append(f)
        return texts, targets

    vec = DictVectorizer()
    
    train, train_target = seperate(train_data)
    test, test_target   = seperate(test_data)

    vec.fit(train)
    
    train_vec = vec.transform(train)
    test_vec  = vec.transform(test)

    clf = LogisticRegression(random_state=0, solver='lbfgs', max_iter=200)
    clf.fit(train_vec, train_target)
    
    pred = clf.predict(test_vec)
    score = clf.score(test_vec, test_target)

    return pred, score

In [27]:
accuracies, mean, std = n_fold_n(logisticReg_exp, data, folds=9)
print(accuracies)
print("-")
print(mean, std)

[0.765, 0.84, 0.795, 0.795, 0.83, 0.77, 0.835, 0.81, 0.79]
-
0.8033333333333332 0.027386127875258286


### 1.5.2

In [28]:
common_words = [1000, 2000, 5000, 10000, 20000]

for word_size in common_words:
    accuracies, mean, std = n_fold_n(bernoulli_exp_p, data, feature_numbs=word_size)
    print("The {} most common words, bernoulli:".format(word_size))
    print("Accuracies, bernoulli: ", accuracies)
    print("Mean, bernoulli: ", mean)
    print("Standard deviation, bernoulli: ", std)
    print("-")

print("------------------------")
    
for word_size in common_words:
    accuracies, mean, std = n_fold_n(logisticReg_exp, data, feature_numbs=word_size)
    print("The {} most common words, LogisticRegression:".format(word_size))
    print("Accuracies, LogisticRegression: ", accuracies)
    print("Mean, LogisticRegression: ", mean)
    print("Standard deviation, LogisticRegression: ", std)

The 1000 most common words, bernoulli:
Accuracies, bernoulli:  [0.8, 0.765, 0.785, 0.765, 0.755, 0.77, 0.845, 0.755, 0.75]
Mean, bernoulli:  0.7766666666666667
Standard deviation, bernoulli:  0.030103986446980733
-
The 2000 most common words, bernoulli:
Accuracies, bernoulli:  [0.775, 0.805, 0.77, 0.815, 0.83, 0.795, 0.885, 0.78, 0.8]
Mean, bernoulli:  0.8061111111111111
Standard deviation, bernoulli:  0.03533569174519031
-
The 5000 most common words, bernoulli:
Accuracies, bernoulli:  [0.81, 0.805, 0.74, 0.8, 0.81, 0.815, 0.885, 0.805, 0.835]
Mean, bernoulli:  0.8116666666666666
Standard deviation, bernoulli:  0.03758324094593227
-
The 10000 most common words, bernoulli:
Accuracies, bernoulli:  [0.82, 0.835, 0.77, 0.82, 0.83, 0.82, 0.845, 0.81, 0.84]
Mean, bernoulli:  0.821111111111111
Standard deviation, bernoulli:  0.022189211592823894
-
The 20000 most common words, bernoulli:
Accuracies, bernoulli:  [0.815, 0.82, 0.735, 0.83, 0.785, 0.795, 0.84, 0.795, 0.835]
Mean, bernoulli:  0.80

### Exercise 1.6

In [29]:
(acc, mean, standard) = n_fold_n(logisticReg_exp, movie_test, feature_numbs=word_size)
print("Accuracies for movie_test, LogisticRegression: ", acc)
print("Mean for movie_test, LogisticRegression: ", mean)
print("Standard deviation for movie_test, LogisticRegression: ", standard)

(acc, mean, standard) = n_fold_n(bernoulli_exp_p, movie_test, feature_numbs=word_size)
print("Accuracies for movie_test, Bernoilli: ", acc)
print("Mean for movie_test, Bernoilli: ", mean)
print("Standard deviation for movie_test, Bernoilli: ", standard)

Accuracies for movie_test, LogisticRegression:  [0.5909090909090909, 0.6363636363636364, 0.6363636363636364, 0.5, 0.36363636363636365, 0.45454545454545453, 0.5, 0.45454545454545453, 0.5454545454545454]
Mean for movie_test, LogisticRegression:  0.5202020202020202
Standard deviation for movie_test, LogisticRegression:  0.09122420135448708
Accuracies for movie_test, Bernoilli:  [0.4090909090909091, 0.5, 0.2727272727272727, 0.5, 0.36363636363636365, 0.45454545454545453, 0.5454545454545454, 0.5909090909090909, 0.5909090909090909]
Mean for movie_test, Bernoilli:  0.4696969696969697
Standard deviation for movie_test, Bernoilli:  0.10660035817780524


In [32]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

def seperate(data):
    text = []
    target = []
    for t, f in data:
        text.append(t)
        target.append(f)
    return text, target
_, test_target = seperate(movie_test)

pred, acc = logisticReg_exp(movie_dev, movie_test, feature_numbs=20000)

print("Acc:", acc)
print('F-score:', f1_score(test_target, pred, average="macro"))
print('Precision:', precision_score(test_target, pred, average="macro"))
print('Recall:', recall_score(test_target, pred, average="macro"))

pred, acc = bernoulli_exp_p(movie_test, movie_test, feature_numbs=10000)

print("Acc:", acc)
print('F-score:', f1_score(test_target, pred, average="macro"))
print('Precision:', precision_score(test_target, pred, average="macro"))
print('Recall:', recall_score(test_target, pred, average="macro"))


Acc: 0.54
F-score: 0.5392628205128205
Precision: 0.5393483709273184
Recall: 0.539285356821139
Acc: 0.515
F-score: 0.33993399339933994
Precision: 0.2575
Recall: 0.5


I chose to use LogisticRegression with feature_numbs=20000 and Bernoulli with feature_numbs=10000
This yielded the best results